In [23]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [24]:

client = oandapyV20.API(access_token=access_token)
currency_pair = "EUR_USD"
amount_per_trade = 100
count = 1000
interval = 10
window_size = 50

In [25]:
def calculate_units(price, account_balance, amount_per_trade):
    # calculate the units to buy/sell based on the amount of money you want to trade and the account balance
    if account_balance < amount_per_trade:
        print("Insufficient balance.")
        return None
    return round(amount_per_trade / float(price))

In [26]:
def buy(price, units_to_buy):
    price = float(price)
    margin_rate = 0.033
    stop_loss_price = round(price - (price * (margin_rate / 100)),5)
    take_profit_price = round(price + (price * (margin_rate / 100)),5)
    data = {
        "order": {
            "price": str(price),
            "stopLossOnFill": {
                "timeInForce": "GTC",
                "price": str(stop_loss_price),
            },
            "takeProfitOnFill": {
                "timeInForce": "GTC",
                "price": str(take_profit_price),
            },
            "instrument": currency_pair,
            "units": str(units_to_buy),
            "type": "MARKET",
            "positionFill": "DEFAULT"
        }
    }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r)

In [27]:
def sell(price, units_to_sell):
    price = float(price)
    margin_rate = 0.33333333333333
    stop_loss_price = round(price + (price * (margin_rate / 100)),5)
    take_profit_price = round(price - (price * (margin_rate / 100)),5)
    data = {
        "order": {
            "price": str(price),
            "stopLossOnFill": {
                "timeInForce": "GTC",
                "price": str(stop_loss_price)
            },
            "takeProfitOnFill": {
                "timeInForce": "GTC",
                "price": str(take_profit_price)
            },
            "instrument": currency_pair,
            "units": str(units_to_sell),
            "type": "MARKET",
            "positionFill": "DEFAULT"
        }
    }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r)


In [28]:
def get_historical_data(count):
    r = instruments.InstrumentsCandles(instrument=currency_pair, params={"count":count, 'granularity': 'H4'})
    data = client.request(r)
    close_prices = [candle["mid"]["c"] for candle in data["candles"]]
    return close_prices


In [29]:
def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0,1))
    data_normalized = scaler.fit_transform(np.array(data).reshape(-1,1))
    return data_normalized, scaler

In [30]:
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data)-window_size-1):
        X.append(data[i:(i+window_size), 0])
        y.append(data[i + window_size, 0])
    return np.array(X), np.array(y)


In [31]:
def train_model(X_train, y_train):
    model = models.Sequential()
    model.add(layers.LSTM(50, return_sequences=True, input_shape=(window_size, 1)))
    model.add(layers.LSTM(50, return_sequences=False))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=10,validation_split=0.35)
    return model

In [32]:
def main_loop():
    while True:
        try:
            print("Fetching historical data...")
            data = get_historical_data(count)

            print("Normalizing data...")
            data_normalized, scaler = normalize_data(data)

            print("Creating dataset...")
            X, y = create_dataset(data_normalized, window_size)

            print("Splitting dataset into training and test sets...")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            print("Reshaping input data for the neural network...")
            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

            print("Training model...")
            model = train_model(X_train, y_train)

            print("Making prediction on test set...")
            prediction = model.predict(X_test)
            prediction_unscaled = scaler.inverse_transform(prediction)

            print("Getting current price...")
            r = instruments.InstrumentsCandles(instrument=currency_pair, params={"count": 1, "granularity": "M1"})
            dataC = client.request(r)
            current_price = float(dataC["candles"][0]["mid"]["c"])

            print("Getting account details...")
            r = accounts.AccountDetails(accountID=account_id)
            account_details = client.request(r)
            account_balance = round(float(account_details['account']['balance']))

            predicted_price = prediction_unscaled[-1][0]
            print(f"Current price: {current_price}, predicted price: {predicted_price}")

            if predicted_price > current_price:
                print("Predicted price is greater than current price, deciding to buy...")
                units_to_buy = calculate_units(current_price, account_balance, amount_per_trade)
                if units_to_buy:
                    print(f"Buying {units_to_buy} units...")
                    buy(current_price, units_to_buy)
            elif predicted_price < current_price:
                print("Predicted price is less than current price, deciding to sell...")
                units_to_sell = calculate_units(current_price, account_balance, amount_per_trade)
                if units_to_sell:
                    print(f"Selling {units_to_sell} units...")
                    sell(current_price, units_to_sell)

            print(f"Waiting {interval} seconds before next iteration...")
            time.sleep(interval)
        except Exception as e:
            print(f"Error encountered: {e}")
            break


In [33]:
main_loop()

Fetching historical data...
Normalizing data...
Creating dataset...
Splitting dataset into training and test sets...
Reshaping input data for the neural network...
Training model...
Epoch 1/50
50/50 [==============================] - 3s 23ms/step - loss: 0.0265 - val_loss: 0.0048
Epoch 2/50
50/50 [==============================] - 1s 15ms/step - loss: 0.0022 - val_loss: 0.0058
Epoch 3/50
50/50 [==============================] - 1s 14ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 4/50
50/50 [==============================] - 1s 14ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 5/50
50/50 [==============================] - 1s 14ms/step - loss: 0.0017 - val_loss: 0.0020
Epoch 6/50
50/50 [==============================] - 1s 14ms/step - loss: 0.0018 - val_loss: 0.0022
Epoch 7/50
50/50 [==============================] - 1s 15ms/step - loss: 0.0016 - val_loss: 0.0021
Epoch 8/50
50/50 [==============================] - 1s 17ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 9/50
50/50 [========